In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service = service, options = options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[1].text
print("목표 키워드 :", target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색 완료. 현재 페이지 타이틀 :", driver.title)

scroll_pause = 2

for _ in range(10) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)

products = driver.find_elements(By.CSS_SELECTOR, "div.sc-d36st-1.elXjMR")[:100]
products_data = []

for product in products :
    try:
        name = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip()
        url = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href")
        review_info = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text
        # print(name,url,review_info)
        review_count = int(re.sub(r"[^\d]","", review_info)) if review_info else 0
        products_data.append({"상품명":name,"URL":url,"리뷰수":review_count})
    except:
        continue

df = pd.DataFrame(products_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True)

driver.quit()
df_top30.head()

목표 키워드 : 이자반
검색 완료. 현재 페이지 타이틀 : 이자반 | 무신사 추천 상품


,상품명,URL,리뷰수
0,이자반X탄산 링거 티셔츠 스카이 블루 (맨),https://www.musinsa.com/products/5255001,12
1,이자반X탄산 고등어 티셔츠 화이트 (맨),https://www.musinsa.com/products/5254998,7
2,이자반X탄산 링거 티셔츠 스카이 블루 (우먼),https://www.musinsa.com/products/5254991,5
3,이자반X탄산 링거 티셔츠 와인 (맨),https://www.musinsa.com/products/5255003,5
4,이자반X탄산 고등어 티셔츠 화이트 (우먼),https://www.musinsa.com/products/5254988,4


In [3]:
from openpyxl import Workbook

driver = webdriver.Chrome(service=service, options=options)
all_reviews = []

for idx, row in df_top30.iterrows() :
    driver.get(row['URL'])
    time.sleep(2)

    try:
        review_tab = driver.find_element(By.XPATH, "//button[@data-button-id='prd_review_tab']")
        review_tab.click()
        time.sleep(2)

        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)

        review_elements = driver.find_elements(By.CSS_SELECTOR, ".text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text :
                all_reviews.append({
                    "상품명":row["상품명"],
                    "리뷰":text
                })
    
    except Exception as e :
        print("리뷰 수집 실패: ",e)
        continue
driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"

ws.append(["상품명", "리뷰"])

for r in all_reviews :
    ws.append([r["상품명"],r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("저장 완료")

저장 완료


In [20]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

EXCEL_PATH = "musinsa_top30_reviews.xlsx"
DB_NAME = "musinsa_db_v4"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = "tlgus01937"
HOST = "localhost"
PORT = 3306

df = pd.read_excel(EXCEL_PATH)

conn = pymysql.connect(host = HOST, user = USER, password = PASSWORD, port = PORT)
cursor = conn.cursor()
cursor.execute(
    f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)
print(f"데이터가 MySQL [{DB_NAME}]의 테이블 [{TABLE_NAME}]에 성공적으로 저장되었습니다.")

데이터가 MySQL [musinsa_db_v4]의 테이블 [reviews]에 성공적으로 저장되었습니다.


In [9]:
import pymysql

try:
    conn = pymysql.connect(
        host="127.0.0.1",
        user="root",
        password="Whysolonely04@",
        database="musinsa_db",
        port=3306
    )
    print("✅ MySQL 연결 성공")
    conn.close()
except Exception as e:
    print("❌ MySQL 연결 실패:", e)
print(engine_str)

✅ MySQL 연결 성공
mysql+pymysql://root:Whysolonely04@@127.0.0.1:3306/musinsa_db?charset=utf8mb4


In [5]:
pip install gspread

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0/6 [rsa]
   ---------------------------------------- 0/6 [rsa]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------------- ------------- 4/6 [google-auth-oauthlib]
   ---------------------------------------- 6/6 [gspread]



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
pip install oauth2client

Defaulting to user installation because normal site-packages is not writeable

   -------------------- ------------------- 1/2 [oauth2client]
   -------------------- ------------------- 1/2 [oauth2client]
   ---------------------------------------- 2/2 [oauth2client]

Note: you may need to restart the kernel to use updated packages.


In [5]:
from konlpy.tag import Okt
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

df = pd.read_excel("musinsa_top30_reviews.xlsx")

okt = Okt()
df["리뷰_키워드"] = df["리뷰"].apply(lambda x: ", ".join(okt.nouns(str(x))))
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("deep-ethos-468703-n8-1bfd424a02c2.json", scope)
client = gspread.authorize(creds)


spreadsheet = client.open("무신사 리뷰 분석_v1")
sheet1 = spreadsheet.sheet1
sheet1.update([df.columns.values.tolist()] + df.values.tolist())

try :
    spreadsheet.add_worksheet(title="키워드 분석", rows="1000", cols = "20")
except: 
    pass

sheet2 = spreadsheet.worksheet("키워드 분석")
sheet2.clear()
sheet2.update([["상품명","리뷰","리뷰_키워드"]]+df[["상품명","리뷰","리뷰_키워드"]].values.tolist())
print("😴구글 스프레드시트에 원본 + 키워드 분석자료 업데이트 완료")

😴구글 스프레드시트에 원본 + 키워드 분석자료 업데이트 완료
